In [1]:
flights=sc.textFile("file:///E:/Eclipse_Android_workspaces/ApacheSpark/data/FlightsData/flights.csv")
airlines=sc.textFile("file:///E:/Eclipse_Android_workspaces/ApacheSpark/data/FlightsData/airlines.csv")
airports=sc.textFile("file:///E:/Eclipse_Android_workspaces/ApacheSpark/data/FlightsData/airports.csv")

In [2]:
from datetime import datetime
from collections import namedtuple
fields=('date','airline','flightnum','origin','dest','dep','dep_delay','arv','arv_delay','airtime','distance')
Flight=namedtuple('Flight',fields,verbose=True)
DATE_FMT="%Y-%m-%d"
TIME_FMT="%H%M"
def parse(row):
    row[0]=datetime.strptime(row[0],DATE_FMT).date()
    row[5]=datetime.strptime(row[5],TIME_FMT).time()
    row[6]=float(row[6])
    row[7]=datetime.strptime(row[7],TIME_FMT).time()
    row[8]=float(row[8])
    row[9]=float(row[9])
    row[10]=float(row[10])
    return Flight(*row[:11])

from builtins import property as _property, tuple as _tuple
from operator import itemgetter as _itemgetter
from collections import OrderedDict

class Flight(tuple):
    'Flight(date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance)'

    __slots__ = ()

    _fields = ('date', 'airline', 'flightnum', 'origin', 'dest', 'dep', 'dep_delay', 'arv', 'arv_delay', 'airtime', 'distance')

    def __new__(_cls, date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance):
        'Create new instance of Flight(date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance)'
        return _tuple.__new__(_cls, (date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance))

    @classmethod
    def _make(cls, iterable, new=tuple.__new__, len=len):
        'Make a new Flight object from a sequence or iterable'
        result = new(cls, iterable)
        if len(result) != 11:
  

In [3]:
flightsParsed=flights.map(lambda x:x.split(',')).map(parse)

In [4]:
flightsParsed.persist()

PythonRDD[6] at RDD at PythonRDD.scala:48

In [5]:
totalDistance=flightsParsed.map(lambda x:x.distance).reduce(lambda x,y:x+y)

In [6]:
totalDistance/flightsParsed.count()

794.8585013871385

In [7]:
(flightsParsed.filter(lambda x:x.dep_delay>0).count()/flightsParsed.count())*100

37.53871510922012

In [8]:
sumCount=flightsParsed.map(lambda x:x.dep_delay).aggregate((0,0),
                                                           (lambda acc,value:(acc[0]+value,acc[1]+1)),
                                                           (lambda acc1,acc2:(acc1[0]+acc2[0],acc1[1]+acc2[1])))
sumCount[0]/sumCount[1]

8.313877046894298

In [9]:
delays=flightsParsed.map(lambda x:int(x.dep_delay/60)).countByValue()
delays

defaultdict(int,
            {0: 452963,
             1: 16016,
             2: 4893,
             3: 1729,
             4: 701,
             5: 249,
             6: 113,
             7: 66,
             8: 43,
             9: 26,
             10: 15,
             11: 12,
             12: 9,
             13: 15,
             14: 13,
             15: 4,
             17: 2,
             20: 4,
             21: 3,
             24: 3,
             25: 1,
             28: 1})

In [10]:
flightsParsed.first()

Flight(date=datetime.date(2014, 4, 1), airline='19805', flightnum='1', origin='JFK', dest='LAX', dep=datetime.time(8, 54), dep_delay=-6.0, arv=datetime.time(12, 17), arv_delay=2.0, airtime=355.0, distance=2475.0)

In [11]:
airportDelays=flightsParsed.map(lambda x:(x.origin,x.dep_delay))

In [24]:
"""sumDelays=airportDelays.reduceByKey(lambda x,y:x+y)
countDelays=airportDelays.mapValues(lambda x:1).reduceByKey(lambda x,y:x+y)
avgProcess=sumDelays.join(countDelays)
avg=avgProcess.mapValues(lambda x:x[0]/float(x[1]))
avg.take(5)"""

[('LAX', 8.14151097554333),
 ('BOS', 5.324795523030564),
 ('SFO', 9.23209402025279),
 ('ATL', 8.471295966881206),
 ('DCA', 4.645011998628728)]

In [12]:
airportSumCount=airportDelays.combineByKey((lambda value:(value,1)),
                                          (lambda acc,value:(acc[0]+value,acc[1]+1)),
                                           (lambda acc1,acc2:(acc1[0]+acc2[0],acc1[1]+acc2[1])))
avg=airportSumCount.mapValues(lambda x:x[0]/float(x[1]))
avg.take(5)

[('LAX', 8.14151097554333),
 ('BOS', 5.324795523030564),
 ('SFO', 9.23209402025279),
 ('ATL', 8.471295966881206),
 ('DCA', 4.645011998628728)]

In [13]:
topAirports=avg.sortBy(lambda x:-x[1])
topAirports.take(10)

[('PPG', 56.25),
 ('EGE', 32.0),
 ('OTH', 24.533333333333335),
 ('LAR', 18.892857142857142),
 ('RDD', 18.55294117647059),
 ('MTJ', 18.363636363636363),
 ('PUB', 17.54),
 ('EWR', 16.478549005929544),
 ('CIC', 15.931034482758621),
 ('RST', 15.6993006993007)]

In [26]:
import csv
def spilt(line):
    line=line.replace("\"","")
    splits=line.split(",")
    return (splits[0],splits[1])
def notHeader(row):
    return "Description" not in row
airportsRdd=airports.filter(notHeader).map(spilt)
airportsRdd.lookup('LAX')

['Los Angeles']

In [29]:
##SPARK-5063 -we cannot use lookup inside transformation.
##use broadcast variables instead
##avgAirportDelaysDesc=avg.map(lambda x:(airportsRdd[x[0]],x[1]))
airportsRddBc=sc.broadcast(airportsRdd.collectAsMap())
airportsRddBc.value['LAX']
avgAirportDelaysDesc=topAirports.map(lambda x:(airportsRddBc.value[x[0]],x[1]))
avgAirportDelaysDesc.take(10)

[('Pago Pago', 56.25),
 ('Eagle', 32.0),
 ('North Bend/Coos Bay', 24.533333333333335),
 ('Laramie', 18.892857142857142),
 ('Redding', 18.55294117647059),
 ('Montrose/Delta', 18.363636363636363),
 ('Pueblo', 17.54),
 ('Newark', 16.478549005929544),
 ('Chico', 15.931034482758621),
 ('Rochester', 15.6993006993007)]